In [1]:
#Here we import the required libraries:
#df.to_csv('path',index=False)
from IPython.core.display import display, HTML
import warnings
warnings.filterwarnings("ignore")
display(HTML("<style>.container { width:100% !important; }</style>"))


from numpy import arange
import math
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os   
import math

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from sklearn import tree
from scipy.stats import sem


from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RepeatedStratifiedKFold  
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.metrics import r2_score

from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LassoCV

from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler

#Here we define a score function using stratified cross fold validation.


def scores(x):
    lin=i()
    cv = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)
    score = cross_val_score(lin, x.iloc[:,2:], x['vb'], cv=cv)
    s_min.append(score.min())
    s_m.append(score.mean())
    s_max.append(score.max())
    s_var.append(score.var())

    #Checking scores by using our function:
    algos=[LinearRegression,KNeighborsRegressor,RandomForestRegressor,Lasso,ElasticNet,DecisionTreeRegressor,GradientBoostingRegressor]
    s_min=[]
    s_m=[]
    s_max=[]
    s_var=[]
    for i in algos:
         scores(i)


    models = pd.DataFrame({'Method': ['LinearRegression','KNeighborsRegression','RandomForestRegressor','Lasso','ElasticNet','DecisionTreeRegressor',
                                      'GradientBoostingRegressor'],
                           'Test_Score_Min':[s_min[0],s_min[1],s_min[2],s_min[3],s_min[4],s_min[5],s_min[6]],
                           'Test_Score_Mean':[s_m[0],s_m[1],s_m[2],s_m[3],s_m[4],s_m[5],s_m[6]],
                           'Test_Score_Max':[s_max[0],s_max[1],s_max[2],s_max[3],s_max[4],s_max[5],s_max[6]],
                           'Test_Score_Var':[s_var[0],s_var[1],s_var[2],s_var[3],s_var[4],s_var[5],s_var[6]]})
    models=models.sort_values(by='Test_Score_Mean',ascending=False)
    print(list(x.columns))
    print(len(list(x.columns)))
    return models

def y_pred(df,est):

    df1=df.copy()
    X=df1.iloc[:,1:]
    
    y=df1['vb']
    

    stf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
    
    y_pred = cross_val_predict(est, X, y,cv=stf)
    score=cross_val_score(est, X, y,cv=stf)
    dataset = pd.DataFrame({'vb_real': y, 'vb_prediction': y_pred}, columns=['vb_real', 'vb_prediction'])
    results_mean = dataset.groupby(['vb_real'],as_index=False).mean()
    results_min = dataset.groupby(['vb_real'],as_index=False).min()
    results_max = dataset.groupby(['vb_real'],as_index=False).max()
    df1_result=pd.concat([results_mean,results_min,results_max],keys=[' Lasso Mean of bubbles', 'Lasso Min of bubbles','Lasso Max of bubbles'],axis=1)
    print(score.mean())

    return df1_result

def groupby(y,y_pred):
    global dataset, dg
    dataset = pd.DataFrame({'vb_real': y, 'vb_prediction': y_pred}, columns=['vb_real', 'vb_prediction'])
    results_mean = dataset.groupby(['vb_real'],as_index=False).mean()
    results_min = dataset.groupby(['vb_real'],as_index=False).min()
    results_max = dataset.groupby(['vb_real'],as_index=False).max()
    results_std = dataset.groupby(['vb_real'],as_index=False).std()
    dg=pd.concat([results_min['vb_prediction'],results_mean['vb_prediction'],results_max['vb_prediction']],keys=['Min', 'Mean', 'Max'],axis=1)
    return dg




def split(df,train_size):       # This is same as StratifiedKFold method takes n% test size from each label of dataset.
    global X_train, X_test, y_test, y_train,test_x,train_x
    Y=df['vb']
    yls=np.array(Y)
    yls_val = list(set(yls))
    train_x = pd.concat([df[df['vb']==i].sample(frac=train_size, replace=False) for i in yls_val])
    X_train = train_x.iloc[:,1:]               #resulting x train
    y_train=train_x['vb']
    test_x = df.loc[~df.index.isin(train_x.index)]
    X_test=test_x.iloc[:,1:]
    y_test=test_x['vb']
    print(' split(df,train_size) function applies  ShuffledStratifiedKFold cross validation and returns X_test ,X_train, y_test, y_train ')
    return     


def box_plot(dataset1,dataset2):
    dataset1['% deviations']=((dataset1['vb_prediction']-dataset1['vb_real'])/dataset1['vb_real'])*100
    dataset2['% deviations']=((dataset2['vb_prediction']-dataset2['vb_real'])/dataset2['vb_real'])*100
    import plotly.graph_objects as go

    x = dataset1['vb_real']

    fig = go.Figure()
    fig.add_trace(go.Box(
        y=dataset1['% deviations'],
        x=x,
        name='kale',
        marker_color='#3D9970'
    ))
    fig.add_trace(go.Box(
        y=dataset2['% deviations'],
        x=x,
        name='radishes',
        marker_color='#FF4136'
    ))

    fig.update_xaxes(type="category")

    fig.update_layout( 
        xaxis = dict(tickmode = 'auto', dtick = 5, title="bubble volume[µL]"),
        yaxis = dict(title=" relative devition[%]"),
        boxmode='group'  # group together boxes of the different traces for each value of x,
    )

    fig.show()
    

from sklearn.metrics import mean_squared_error
def root_mean_squared_error(y_true, y_pred):
    ''' Root mean squared error regression loss
    
    Parameters
    ----------
    y_true : array-like of shape = (n_samples) or (n_samples, n_outputs)
    Ground truth (correct) target values.

    y_pred : array-like of shape = (n_samples) or (n_samples, n_outputs)
    Estimated target values.
    '''
    return np.sqrt(mean_squared_error(y_true, y_pred))



In [2]:
df=pd.read_csv('all_250t_250n_max_min_std_avg_78_features.csv')
df=df.iloc[:,:]
df

,vb,min_all_avg,max_all_avg,mean_all_avg,median_all_avg,time_to_Peak_all_avg,variance_all_avg,rms_all_avg,energy_simpsons_all_avg,crest_factor_all_avg,...,svd_entropy_all_250n_avg,cid_ce_all_250n_avg,app_entropy_all_250n_avg,app_entropy_all_250t_avg,sample_entropy_all_250n_avg,sample_entropy_all_250t_avg,benford_correlation_all_250t_avg,benford_correlation_all_250n_avg,min_all_250n_avg,min_all_250t_avg
0,5.0,-8.884786,18.013173,1.121821,0.390724,246.081633,130.926580,5.196627,896.540597,3.429126,...,0.989924,22.430463,0.998827,1.009635,1.700297,1.668619,0.771037,0.795747,-5.599072,-5.456215
1,5.0,-8.525532,16.188754,0.542835,0.280590,257.183673,38.438620,3.950886,435.254564,3.640311,...,0.993032,23.072155,1.015609,1.019815,1.620138,1.653597,0.785080,0.786250,-5.709206,-5.484716
2,5.0,-7.014213,10.842930,0.375405,0.210277,237.653061,17.663462,3.177249,299.684796,3.159494,...,0.995424,22.882751,1.013498,1.012926,1.641626,1.657901,0.793707,0.791318,-5.565233,-5.891764
3,5.0,-6.170402,15.727557,0.771562,0.441843,228.897959,50.928493,3.845624,616.637645,3.035497,...,0.995611,23.379019,1.014707,1.020296,1.619902,1.698297,0.778885,0.775844,-5.313259,-5.333668
4,5.0,-7.955254,13.861073,0.520818,0.299848,251.306122,26.020647,3.645854,415.628219,3.413758,...,0.992663,22.950719,0.998641,1.011205,1.673978,1.643214,0.806397,0.785757,-5.587907,-5.853213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,900.0,-53.846611,416.541144,53.905302,-0.581305,295.693878,52030.242876,142.085187,43125.751745,3.426954,...,0.931820,19.122217,0.883004,0.797100,1.504239,1.276669,0.765879,0.778891,-8.622121,-16.071101
348,900.0,-54.105824,410.649278,52.118998,-0.320110,383.142857,45348.131648,135.524884,41694.792012,3.532173,...,0.928225,19.513433,0.916231,0.839651,1.462180,1.363634,0.810487,0.788090,-7.616028,-15.024191
349,900.0,-50.675751,414.956902,51.504785,-0.420649,348.326531,49000.747762,138.199145,41203.622544,3.563863,...,0.910223,19.820555,0.927078,0.912389,1.448568,1.394955,0.800069,0.777357,-9.083914,-10.961465
350,900.0,-52.202568,410.021922,52.065595,-0.049507,357.979592,45564.230598,137.627301,41653.816515,3.447565,...,0.906716,19.341111,0.919419,0.882263,1.455675,1.344003,0.771693,0.762539,-8.590323,-12.386241


In [13]:
def las1():
    global search1, search2, search3, y_te_1, y_te_2,y_te_3, l1,l2,l3
    

    # loading libraries
    from sklearn import decomposition, datasets
    from sklearn import linear_model
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import GridSearchCV, cross_val_score
    from sklearn.preprocessing import StandardScaler

    # Creating an scaler object
    std_slc = StandardScaler()

    # Creating a linear regression object with an L2 penalty
    lasso = linear_model.Lasso()

    # Creating a pipeline of three steps. First, standardize the data.
    # Second, tranform the data with PCA.
    # Third, train a Decision Tree Classifier on the data.
    pipe = Pipeline(steps=[('std_slc', std_slc),('lasso', lasso)])


    # Creating lists of parameter for Lasso Regression
    alpha = arange(0,1, 0.001)
    # Creating a dictionary of all the parameter options 
    # Note has you can access the parameters of steps of a pipeline by using '__’
    parameters = dict(lasso__alpha=alpha)

    # Conducting Parameter Optmization With Pipeline
    # Create a grid search object

    search1 = GridSearchCV(pipe, parameters,cv=5, n_jobs=-1)
    search2 = GridSearchCV(pipe, parameters,cv=5, n_jobs=-1)
    search3 = GridSearchCV(pipe, parameters,cv=5, n_jobs=-1)
    
    
def elastic1():
    global search4, search5, search6, y_te_4, y_te_5,y_te_6, l4,l5,l6


    # loading libraries
    from sklearn import decomposition, datasets
    from sklearn import linear_model
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import GridSearchCV, cross_val_score
    from sklearn.preprocessing import StandardScaler

    # Creating an scaler object
    std_slc = StandardScaler()

    # Creating a linear regression object with an L2 penalty
    elastic = linear_model.ElasticNet()

    # Creating a pipeline of three steps. First, standardize the data.
    # Second, tranform the data with PCA.
    # Third, train a Decision Tree Classifier on the data.
    pipe = Pipeline(steps=[('std_slc', std_slc),('elastic', elastic)])


    # Creating lists of parameter for Lasso Regression
    alpha = arange(0,1, 0.001)
    # Creating a dictionary of all the parameter options 
    # Note has you can access the parameters of steps of a pipeline by using '__’
    parameters = dict(elastic__alpha=alpha)

    # Conducting Parameter Optmization With Pipeline
    # Create a grid search object

    search4 = GridSearchCV(pipe, parameters,cv=5, n_jobs=-1)
    search5 = GridSearchCV(pipe, parameters,cv=5, n_jobs=-1)
    search6 = GridSearchCV(pipe, parameters,cv=5, n_jobs=-1)    
    
    
def tree():
    
    global search7, search8, search9, y_te_7, y_te_8,y_te_9, l7,l8,l9
   
    


    # loading libraries
    from sklearn import decomposition, datasets
    from sklearn import tree
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import GridSearchCV, cross_val_score
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import RandomizedSearchCV

    # Creating a DecisionTreeRegressor object
    dtreeReg = tree.DecisionTreeRegressor()

    # Creating a pipeline of three steps. First, standardize the data .
    # Second, tranform the data with PCA.
    # Third, train a Decision Tree Classifier on the data.
    pipe = Pipeline(steps=[('dtreeReg', dtreeReg)])

   

    # Creating lists of parameter for DecisionTreeRegressor
    max_depth = [3,4,5]
    min_samples_leaf = [2,3,4,5,6,7,8,9,10]
    

    # Creating a dictionary of all the parameter options 
    # Note has you can access the parameters of steps of a pipeline by using '__’
    parameters = dict(dtreeReg__max_depth=max_depth, dtreeReg__min_samples_leaf=min_samples_leaf)

    
    
    #rsf= RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=0)
    
    # Conducting Parameter Optmization With Pipeline
    # Creating a grid search object
    search7 = GridSearchCV(pipe, parameters,cv=5, n_jobs=-1)
    search8 = GridSearchCV(pipe, parameters,cv=5, n_jobs=-1)
    search9 = GridSearchCV(pipe, parameters,cv=5, n_jobs=-1)    
    
    
def forest():
    
    global search10, search11, search12, y_te_10, y_te_11,y_te_12, l10,l11,l12
    
    

    # loading libraries
    from sklearn import decomposition, datasets
    from sklearn import tree
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import GridSearchCV, cross_val_score
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.ensemble import RandomForestRegressor

    # Creating a DecisionTreeRegressor object
  
    rf = RandomForestRegressor()

    # Creating a pipeline of three steps. First, standardize the data .
    # Second, tranform the data with PCA.
    # Third, train a Decision Tree Classifier on the data.
    pipe = Pipeline(steps=[('rf', rf )])

    # Creating Parameter Space
    # Creating a list of a sequence of integers from 1 to 30 (the number of features in X + 1)
   

    # Creating lists of parameter for DecisionTreeRegressor
    # Maximum number of levels in tree
    max_depth = [3,4,5]
    min_samples_leaf = list(arange(3,15))
    min_samples_split=list(arange(2,11))
    # Number of trees in random forest
    n_estimators = [100]
    
    # Creating a dictionary of all the parameter options 
    # Note has you can access the parameters of steps of a pipeline by using '__’
    parameters = dict( rf__max_depth=max_depth, rf__min_samples_leaf=min_samples_leaf, rf__n_estimators=n_estimators,rf__min_samples_split=min_samples_split)

    
   # Fitting the grid search
    
    search10 = GridSearchCV(pipe, parameters,cv=5, verbose=2, n_jobs = -1)
    search11 = GridSearchCV(pipe, parameters,cv=5,verbose=2, n_jobs = -1)
    search12 = GridSearchCV(pipe, parameters,cv=5,verbose=2, n_jobs = -1)    
    
def xgbr():
    
    global search13, search14, search15, y_te_13, y_te_14,y_te_15, l13,l14,l15
 
    import numpy as np
    import pandas as pd
    from sklearn import preprocessing
    from xgboost.sklearn import XGBRegressor
    import datetime
    from sklearn.model_selection import GridSearchCV

    now = datetime.datetime.now()


    # Various hyper-parameters to tune
    xgb1 = XGBRegressor()
    parameters = { 'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [3,4,5],
              'min_child_weight': list(arange(1,17)),
              'min_split_loss': list(arange(100,520,20)),
              'subsample': [0.3,0.4,0.5,0.6,0.7,0.8]}

    search13 = GridSearchCV(xgb1,parameters,cv = 5,n_jobs = -1,verbose=True)
    search14 = GridSearchCV(xgb1,parameters,cv = 5,n_jobs = -1,verbose=True)
    search15 = GridSearchCV(xgb1,parameters,cv = 5,n_jobs = -1,verbose=True)    

In [16]:
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
l6=[]
l7=[]
l8=[]
l9=[]
l10=[]
l11=[]
l12=[]
l13=[]
l14=[]
l15=[]


for i in range(0,5):

    split(df,0.8)

    
    las1()
    search1.fit(X_train.iloc[:,:40], y_train)
    search2.fit(X_train.iloc[:,:58], y_train)
    search3.fit(X_train.iloc[:,:78], y_train)
    
    elastic1()
    search4.fit(X_train.iloc[:,:40], y_train)
    search5.fit(X_train.iloc[:,:58], y_train)
    search6.fit(X_train.iloc[:,:78], y_train)
    
    tree()
    search7.fit(X_train.iloc[:,:40], y_train)
    search8.fit(X_train.iloc[:,:58], y_train)
    search9.fit(X_train.iloc[:,:78], y_train)
    
    forest()
    search10.fit(X_train.iloc[:,:40], y_train)
    search11.fit(X_train.iloc[:,:58], y_train)
    search12.fit(X_train.iloc[:,:78], y_train)
    
    xgbr()
    search13.fit(X_train.iloc[:,:40], y_train)
    search14.fit(X_train.iloc[:,:58], y_train)
    search15.fit(X_train.iloc[:,:78], y_train)
    
    
    
    rmse_te_1=root_mean_squared_error(y_test, search1.predict(X_test.iloc[:,:40]))
    rmse_tr_1=root_mean_squared_error(y_train, search1.predict(X_train.iloc[:,:40]))
    y_te_1= search1.predict(X_test.iloc[:,:40])
    rmse_te_2=root_mean_squared_error(y_test, search2.predict(X_test.iloc[:,:58]))
    rmse_tr_2=root_mean_squared_error(y_train, search2.predict(X_train.iloc[:,:58]))
    y_te_2= search2.predict(X_test.iloc[:,:58]) 
    rmse_te_3=root_mean_squared_error(y_test, search3.predict(X_test.iloc[:,:78]))
    rmse_tr_3=root_mean_squared_error(y_train, search3.predict(X_train.iloc[:,:78]))
    y_te_3= search3.predict(X_test.iloc[:,:78])
    
    rmse_te_4=root_mean_squared_error(y_test, search4.predict(X_test.iloc[:,:40]))
    rmse_tr_4=root_mean_squared_error(y_train, search4.predict(X_train.iloc[:,:40]))
    y_te_4= search4.predict(X_test.iloc[:,:40])
    rmse_te_5=root_mean_squared_error(y_test, search5.predict(X_test.iloc[:,:58]))
    rmse_tr_5=root_mean_squared_error(y_train, search5.predict(X_train.iloc[:,:58]))
    y_te_5= search5.predict(X_test.iloc[:,:58]) 
    rmse_te_6=root_mean_squared_error(y_test, search6.predict(X_test.iloc[:,:78]))
    rmse_tr_6=root_mean_squared_error(y_train, search6.predict(X_train.iloc[:,:78]))
    y_te_6= search6.predict(X_test.iloc[:,:78])
    
    rmse_te_7=root_mean_squared_error(y_test, search7.predict(X_test.iloc[:,:40]))
    rmse_tr_7=root_mean_squared_error(y_train, search7.predict(X_train.iloc[:,:40]))
    y_te_7= search7.predict(X_test.iloc[:,:40])
    rmse_te_8=root_mean_squared_error(y_test, search8.predict(X_test.iloc[:,:58]))
    rmse_tr_8=root_mean_squared_error(y_train, search8.predict(X_train.iloc[:,:58]))
    y_te_8= search8.predict(X_test.iloc[:,:58]) 
    rmse_te_9=root_mean_squared_error(y_test, search9.predict(X_test.iloc[:,:78]))
    rmse_tr_9=root_mean_squared_error(y_train, search9.predict(X_train.iloc[:,:78]))
    y_te_9= search9.predict(X_test.iloc[:,:78])
    
    rmse_te_10=root_mean_squared_error(y_test, search10.predict(X_test.iloc[:,:40]))
    rmse_tr_10=root_mean_squared_error(y_train, search10.predict(X_train.iloc[:,:40]))
    y_te_10= search10.predict(X_test.iloc[:,:40])
    rmse_te_11=root_mean_squared_error(y_test, search11.predict(X_test.iloc[:,:58]))
    rmse_tr_11=root_mean_squared_error(y_train, search11.predict(X_train.iloc[:,:58]))
    y_te_11= search11.predict(X_test.iloc[:,:58]) 
    rmse_te_12=root_mean_squared_error(y_test, search12.predict(X_test.iloc[:,:78]))
    rmse_tr_12=root_mean_squared_error(y_train, search12.predict(X_train.iloc[:,:78]))
    y_te_12= search12.predict(X_test.iloc[:,:78])
    
            
    rmse_te_13=root_mean_squared_error(y_test, search13.predict(X_test.iloc[:,:40]))
    rmse_tr_13=root_mean_squared_error(y_train, search13.predict(X_train.iloc[:,:40]))
    y_te_13= search13.predict(X_test.iloc[:,:40])
    rmse_te_14=root_mean_squared_error(y_test, search14.predict(X_test.iloc[:,:58]))
    rmse_tr_14=root_mean_squared_error(y_train, search14.predict(X_train.iloc[:,:58]))
    y_te_14= search14.predict(X_test.iloc[:,:58]) 
    rmse_te_15=root_mean_squared_error(y_test, search15.predict(X_test.iloc[:,:78]))
    rmse_tr_15=root_mean_squared_error(y_train, search15.predict(X_train.iloc[:,:78]))
    y_te_15= search15.predict(X_test.iloc[:,:78])
    
    li1=list(y_te_1)
    li2=list(y_te_2)
    li3=list(y_te_3)
    li4=list(y_te_4)
    li5=list(y_te_5)
    li6=list(y_te_6)
    li7=list(y_te_7)
    li8=list(y_te_8)
    li9=list(y_te_9)
    li10=list(y_te_10)
    li11=list(y_te_11)
    li12=list(y_te_12)
    li13=list(y_te_13)
    li14=list(y_te_14)
    li15=list(y_te_15)
    
    
    li1.append(rmse_te_1)
    li1.append(rmse_tr_1)
    li2.append(rmse_te_2)
    li2.append(rmse_tr_2)
    li3.append(rmse_te_3)
    li3.append(rmse_tr_3)
    li4.append(rmse_te_4)
    li4.append(rmse_tr_4)
    li5.append(rmse_te_5)
    li5.append(rmse_tr_5)
    li6.append(rmse_te_6)
    li6.append(rmse_tr_6)
    li7.append(rmse_te_7)
    li7.append(rmse_tr_7)
    li8.append(rmse_te_8)
    li8.append(rmse_tr_8)
    li9.append(rmse_te_9)
    li9.append(rmse_tr_9)
    li10.append(rmse_te_10)
    li10.append(rmse_tr_10)
    li11.append(rmse_te_11)
    li11.append(rmse_tr_11)
    li12.append(rmse_te_12)
    li12.append(rmse_tr_12)
    li13.append(rmse_te_13)
    li13.append(rmse_tr_13)
    li14.append(rmse_te_14)
    li14.append(rmse_tr_14)
    li15.append(rmse_te_15)
    li15.append(rmse_tr_15)
    

    l1.append(li1)
    l2.append(li2)
    l3.append(li3)
    l4.append(li4)
    l5.append(li5)
    l6.append(li6)    
    l7.append(li7)
    l8.append(li8)
    l9.append(li9)
    l10.append(li10)
    l11.append(li11)
    l12.append(li12)    
    l13.append(li13)
    l14.append(li14)
    l15.append(li15)


l1.insert(0,list(y_test))
data=pd.concat([pd.DataFrame(l1),pd.DataFrame(l2),pd.DataFrame(l3),pd.DataFrame(l4),pd.DataFrame(l5),pd.DataFrame(l6),pd.DataFrame(l7),pd.DataFrame(l8),pd.DataFrame(l9),pd.DataFrame(l10),pd.DataFrame(l11),pd.DataFrame(l12),pd.DataFrame(l13),pd.DataFrame(l14),pd.DataFrame(l15)],axis=0)


 split(df,train_size) function applies  ShuffledStratifiedKFold cross validation and returns X_test ,X_train, y_test, y_train 
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   44.4s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   40.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   51.0s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


...

[23:06:23] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:06:23] WARNING: ..\src\learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   49.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  5.0min finished


[23:11:24] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:11:24] WARNING: ..\src\learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  6.3min finished


[23:17:43] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:17:43] WARNING: ..\src\learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [17]:
data

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,5.000000,5.000000,5.000000,5.000000,5.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,700.000000,700.000000,700.000000,900.000000,900.000000,900.000000,900.000000,900.000000,NaN,NaN
1,-23.995726,-16.356190,0.424127,14.546635,1.736667,-1.861955,8.929655,9.227757,2.409505,2.922653,...,692.360698,702.116371,660.382942,974.261690,905.927464,846.441950,993.781070,866.780468,29.332574,27.571854
2,12.697523,-7.323848,-2.667502,-1.148596,-2.398201,4.838327,4.786494,2.411714,0.897033,9.774132,...,678.258741,663.911730,739.062981,796.968880,826.045116,868.543437,871.339053,882.937999,24.133809,28.672825
3,2.060673,11.877368,-12.477928,-11.135747,5.816383,18.592508,-10.095121,15.834216,19.174302,4.577096,...,662.957564,741.814975,725.095365,803.308742,836.465803,882.896345,823.645035,959.315153,26.628438,28.243385
4,-4.970896,-2.482750,-1.494768,-12.161511,-2.621207,3.155111,1.912214,6.411200,19.245301,3.697501,...,681.054798,733.716376,684.659903,900.580880,856.008488,856.727441,831.956516,904.008243,22.584470,29.037250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,7.800162,5.921838,7.297583,5.980520,7.569100,7.241476,11.431052,10.397264,12.293756,9.526074,...,682.045166,687.625793,680.615967,893.810974,891.717163,902.936401,921.967163,897.711548,30.337802,0.009989
1,6.951788,7.385530,5.596478,6.822535,7.167724,10.691855,5.968075,9.216184,9.684932,7.046870,...,705.281067,659.174072,705.564331,874.654602,891.149536,898.414673,907.783936,901.424744,13.073535,0.004993
2,9.667495,6.911593,6.178175,5.954823,9.910942,13.181696,7.731301,15.950731,25.196522,13.904265,...,672.388000,715.168762,817.438477,882.030457,895.306641,919.489441,894.505798,911.085876,21.610715,0.003968
3,3.515472,7.573937,6.990186,8.785959,9.843958,11.364626,10.308897,7.930188,13.574883,11.211787,...,708.233704,821.670654,699.797119,918.994324,881.358948,894.479980,885.717529,891.269775,24.702674,0.013100
